In [1]:
from z3 import *

# Getting Started

Creating variables and solving a system of constraints

In [2]:
x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


Formula/expression simplification

In [3]:
x = Int('x')
y = Int('y')
print(simplify(x + y + 2 *x + 3))
print(simplify(x < y + x + 2))
print(simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))

3 + 3*x + y
Not(y <= -2)
And(x >= 2, 2*x**2 + y**2 >= 3)


I believe the first print is supposed to have prettier formatting (at least for the web), but it doesn't seeming to be working.

In [4]:
x = Int('x')
y = Int('y')
print(x**2 + y++2 >= 1)
set_option(html_mode=False)
print(x**2 + y++2 >= 1)

x**2 + ToReal(y) + 2 >= 1
x**2 + ToReal(y) + 2 >= 1


The output of this can be copy-pasted into a markdown cell to show prettier formatting

In [5]:
set_option(html_mode=True)
print(x**2 + y**2 >= 1)
set_option(html_mode=False)

x<sup>2</sup> + y<sup>2</sup> &ge; 1


x<sup>2</sup> + y<sup>2</sup> &ge; 1

Expressions can be traversed with built-in functions

In [6]:
x = Int('x')
y = Int('y')
n = x + y >= 3
print("num args: ", n.num_args())
print("children: ", n.children())
print("1st child: ", n.arg(0))
print("2nd child: ", n.arg(1))
print("operator: ", n.decl())
print("op name: ", n.decl().name())

num args:  2
children:  [x + y, 3]
1st child:  x + y
2nd child:  3
operator:  >=
op name:  >=


Solving nonlinear polynomial constraints

In [7]:
x = Real('x')
y = Real('y')
solve(x**2 + 0.12 * y**2 > 3, x**3 + y < 5)

[y = 639/128, x = 1/8]


I have confirmed that this point is a solution using desmos

Z3 can represent arbitrarily large integers, rational numbers, and irrational algebraic numbers (a root of a polynomial with integer coefficients). All of these are internally represented precisely.

In [8]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

set_option(precision=30)
print("Solving, and displaying result with 30 decimal places")
solve(x**2 + y**2 == 3, x**3 == 2)

[y = -1.1885280594?, x = 1.2599210498?]
Solving, and displaying result with 30 decimal places
[y = -1.188528059421316533710369365015?,
 x = 1.259921049894873164767210607278?]


# Configuring the Z3 environment with set_option

In [9]:
print(1/3)            # this is a Python float, not a Z3 rational number
print(RealVal(1)/3)   # creates a Z3 real number 1 and divides by 3
print(Q(3, 2))        # creates a Z3 rational with a numerator and denominator

x = Real('x')
print(x + 1/3)
print(x + "1/3")
print(x + 0.25)

0.3333333333333333
1/3
3/2
x + 3333333333333333/10000000000000000
x + 1/3
x + 1/4


Showing rational numbers in decimal notation. The ? indicates that the output is truncated

In [10]:
x = Real('x')
solve(3*x == 1)

[x = 1/3]


In [11]:
set_option(rational_to_decimal=True)  # show rationals in decimal notation
solve(3*x == 1)

set_option(precision=20)  # change number of decimal places
solve(3*x == 1)

set_option(rational_to_decimal=False)
solve(6*x == 37)

[x = 0.333333333333333333333333333333?]
[x = 0.33333333333333333333?]
[x = 37/6]


Below is an unsatisfiable system of constraints

In [12]:
x = Real('x')
solve(x > 4, x < 0)

no solution


# Boolean Logic

Boolean operators in Z3Py are And, Or, Not, Implies, If(if-then-else). Bi-implications are represented with ==

In [13]:
p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[q = True, p = False, r = False]


In [14]:
p = Bool('r')
q = Bool('q')
print(And(p, q, True))
print(simplify(And(p, q, True)))    # Python Boolean constants True and False can be used
print(simplify(And(p, False)))

And(r, q, True)
And(r, q)
False


In [15]:
p = Bool('p')
x = Real('x')
solve(Or(x < 5, x > 10), Or(p, x**2 == 2), Not(p))

[x = -1.41421356237309504880?, p = False]


# Solvers

In [16]:
x = Int('x')
y = Int('y')

s = Solver()
print(s)
print()

s.add(x > 10, y == x + 2)
print(s)
print("Solving constraints in the solver s ...")
print(s.check())
print()

print("Create a new scope ...")
s.push()
s.add(y < 11)
print(s)
print("Solving updated set of constraints ...")
print(s.check())
print()

print("Restoring state ...")
s.pop()
print(s)
print("Solving restored set of constraints ...")
print(s.check())

[]

[x > 10, y == x + 2]
Solving constraints in the solver s ...
sat

Create a new scope ...
[x > 10, y == x + 2, y < 11]
Solving updated set of constraints ...
unsat

Restoring state ...
[x > 10, y == x + 2]
Solving restored set of constraints ...
sat


Solver() method creates a general purpose solver
add() method adds constraints (asserts constraints)
check() solves the system of asserted constraints
    - sat = satisfiable
    - unsat = unsatisfiable
    - unknown = solver failed to solve constraints
push() method creates a new scope by saving current stack of constraints
pop() method removes any assertions added after the push

Z3 fails to solve the below constraints because Z3 can solve nonlinear polynomial constraints but not 2**x

In [17]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print(s.check())

unknown


Traversing constraints and collecting performance statistics

In [18]:
x = Real('x')
y = Real('y')
s = Solver()
s.add(x > 1, y > 1, Or((x + y > 3), x - y < 2))
print("asserted constraints...")
for c in s.assertions():
    print(c)

print(s.check())
print("statistics for the last check method...")
print(s.statistics())
# Traversing statistics
for k, v in s.statistics():
    print("%s : %s" % (k, v))

asserted constraints...
x > 1
y > 1
Or(x + y > 3, x - y < 2)
sat
statistics for the last check method...
(:arith-make-feasible 3
 :arith-max-columns   8
 :arith-max-rows      2
 :arith-upper         4
 :decisions           2
 :final-checks        1
 :max-memory          19.27
 :memory              18.96
 :mk-bool-var         4
 :num-allocs          129756167
 :num-checks          1
 :rlimit-count        5314
 :time                0.01)
decisions : 2
final checks : 1
num checks : 1
mk bool var : 4
arith-upper : 4
arith-make-feasible : 3
arith-max-columns : 8
arith-max-rows : 2
num allocs : 129756167
rlimit count : 5314
max memory : 19.27
memory : 18.96
time : 0.011


We now look at how to inspect models, which are the solutions Z3 finds

In [19]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print(s.check())

m = s.model()
print("x = %s" % m[x])    # m[x] returns the interpretation of x in the model m

print(m.decls())
print("traversing model...")
for d in m.decls():
    print("%s = %s" % (d.name(), m[d]))    # m[d] returns a textual represention of d

sat
x = 3/2
[y, x, z]
traversing model...
y = 2
x = 3/2
z = 0


# Arithmetic

Below are different ways to declare integer and real variables. Z3 will automatically convert integers to reals where necessary

In [20]:
x = Real('x')
y = Int('y')
a, b, c = Reals('a b c')
s, r = Ints("s r")
print(x + y + 1 + (a + s))
print(ToReal(y) + c)    # ToReal(i) casts an integer expression i into a real expression

x + ToReal(y) + 1 + a + ToReal(s)
ToReal(y) + c


Z3 supports all basic arithmetic operations

In [21]:
a, b, c = Ints('a b c')
d, e = Reals('d e')
solve(a > b + 2,
      a == 2*c + 10,
      c + b <= 1000,
      d >= e)

[d = 0, c = 0, b = 0, e = 0, a = 10]


The simplify() method applies simple transformations on Z3 expressions

In [22]:
x, y = Reals('x y')

# Put expressions in sum-of-monomials form
t = simplify((x+y)**3, som=True)
print(t)

# Use power operator
t = simplify(t, mul_to_power=True)
print(t)

x*x*x + 3*x*x*y + 3*x*y*y + y*y*y
x**3 + 3*x**2*y + 3*x*y**2 + y**3


In [23]:
x, y = Reals('x y')
print(simplify(x == y + 2, ':arith-lhs', True))    # Z3 native option names
print(simplify(x == y + 2, arith_lhs=True))        # Z3Py option names

print("\nAll available options:")
help_simplify()

x + -1*y == 2
x + -1*y == 2

All available options:
algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
blast_select_store (bool) eagerly replace all (select (store ..) ..) term by an if-then-else term (default: false)
bv_extract_prop (bool) attempt t

Z3 supports arbitrarily large numbers. Irrational numbers are always shown in decimal notation, but the Z3 internal representation can be extracted with the sexpr() method (s-expression or Lisp-like notation)

In [24]:
x, y = Reals('x y')
solve(x + 10000000000000000000000 == y, y > 20000000000000000)

print(Sqrt(2) + Sqrt(3))
print(simplify(Sqrt(2) + Sqrt(3)))
print(simplify(Sqrt(2) + Sqrt(3)).sexpr())
# The sexpr() method is available for any Z3 expression
print((x + Sqrt(y) * 2).sexpr())

[y = 20000000000000001, x = -9999979999999999999999]
2**(1/2) + 3**(1/2)
3.14626436994197234232?
(root-obj (+ (^ x 4) (* (- 10) (^ x 2)) 1) 4)
(+ x (* (^ y (/ 1.0 2.0)) 2.0))


# Machine Arithmetic

In [25]:
x = BitVec('x', 16)    # creates a bit-vector variable with 16 bits named x
y = BitVec('y', 16)
print(x + 2)
print((x + 2).sexpr())    # Internal representation

# -1 is equal to 65535 for 16-bit integers
print(simplify(x + y - 1))

# Creating bit-vector constants
a = BitVecVal(-1, 16)    # 16-bit vector with value -1
b = BitVecVal(65535, 16)
print(simplify(a == b))

a = BitVecVal(-1, 32)
b = BitVecVal(65535, 32)
# -1 is not equal to 65535 for 32-bit integers
print(simplify(a == b))

x + 2
(bvadd x #x0002)
65535 + x + y
True
False


In Z3Py the operators <, <=, >, >=, /, % and >> are for signed bit-vectors
The unsigned operaotrs are ULT, ULE, UGE, UDiv, URem, and LShR

In [26]:
x, y = BitVecs('x y', 32)    # Create two bit-vectors of size 32

solve (x + y == 2, x > 0, y > 0)

[y = 1, x = 1]


In [27]:
# Bit-wise operators
# & bit-wise and
# / bit-wise or
# ~ bit-wise not
solve(x & y == ~y)

solve(x < 0)

# using unsigned version of <
solve(ULT(x, 0))

[x = 0, y = 4294967295]
[x = 4294967295]
no solution


In [28]:
# >> arithmetic shift right
# << arithmetic shift left
# LShR logical shift right

x, y = BitVecs('x y', 32)

solve(x >> 2 == 3)

solve(x << 2 == 3)

solve(x << 2 == 24)

[x = 12]
no solution
[x = 6]


# Functions

Functions in Z3 have no side-effects and are defined for all values.
In a constraint such as x + y > 3, x and y are called uninterprted constants, as in they can be interpreted as anything that satisfies the constraint x + y > 3. This means that the function + itself is uninterpreted and can also be interpreted as anything that satisfies the constraint.

In [106]:
x = Int('x')    # uninterpreted integer constant
y = Int('y')    # uninterpreted integer constant

# Define a function f that takes an integer type and returns an integer type
# (Types are also known as sorts)
# f :: Int -> Int
f = Function('f', IntSort(), IntSort())

# Constrain f such that:
#    - f applied twice to x returns x
#    - f applied once to x returns some value y that is not equal to x
solve(f(f(x)) == x, f(x) == y, x != y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


The solution (aka interpretation) above should be read as:
* f(1) is 0
* f(a) is 1 for all a not equal 1

In [30]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()

s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())

# Evaluating expressions in the model
m = s.model()
print("f(f(x)) = ", m.evaluate(f(f(x))))
print("f(x)    = ", m.evaluate(f(x)))

sat
f(f(x)) =  0
f(x)    =  1


# Satisfiability and Validity

A formula/constraint F is valid if F always evaluates to true for any assignment of appropriate values. F is therefore valid if and only if the negation of Not(F) is invalid.

In [31]:
# This is a simpler recreation of the Z3Py function prove()
#    - Creates a solver
#    - Adds the negation of the formula
#    - Checks if the negation is unsatisfiable

p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), (Not(q))))
print(demorgan)

def prove(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("proved")
    else:
        print("failed to prove")
        
print("Proving demorgan...")
prove(demorgan)

And(p, q) == Not(Or(Not(p), Not(q)))
Proving demorgan...
proved


# List Comprehension

In [32]:
# Create list [1..5]
print([x + 1 for x in range(5)])

# Create two lists containing 5 integer variables
X = [Int('x%s' % i) for i in range(5)]
Y = [Int('y%s' % i) for i in range(5)]
print(X)
print(Y)

# Create a list containing X[i] + Y[i]
X_plus_Y = [X[i] + Y[i] for i in range(5)]
print(X_plus_Y)

# Create a list containing X[i] > Y[i]
X_gt_Y = [X[i] > Y[i] for i in range(5)]
print(X_gt_Y)

print(And(X_gt_Y))

# Create a 3x3 "matrix" of integer variables
X = [[Int("x_%s_%s" % (i+1, j+1)) for j in range(3)]
      for i in range(3)]

# pp() is similar to print() but it uses Z3Py formatter for
# lists and tuples instead of Python's formatter
pp(X)

[1, 2, 3, 4, 5]
[x0, x1, x2, x3, x4]
[y0, y1, y2, y3, y4]
[x0 + y0, x1 + y1, x2 + y2, x3 + y3, x4 + y4]
[x0 > y0, x1 > y1, x2 > y2, x3 > y3, x4 > y4]
And(x0 > y0, x1 > y1, x2 > y2, x3 > y3, x4 > y4)
[[x_1_1, x_1_2, x_1_3],
 [x_2_1, x_2_2, x_2_3],
 [x_3_1, x_3_2, x_3_3]]


In [33]:
# Z3Py also provides functions for creating vectors of Boolean, Integer, and Real variables
X = IntVector('x', 5)
Y = RealVector('y', 5)
P = BoolVector('p', 5)
print(X)
print(Y)
print(P)
print([y**2 for y in Y])
print(Sum([y**2 for y in Y]))

[x__0, x__1, x__2, x__3, x__4]
[y__0, y__1, y__2, y__3, y__4]
[p__0, p__1, p__2, p__3, p__4]
[y__0**2, y__1**2, y__2**2, y__3**2, y__4**2]
y__0**2 + y__1**2 + y__2**2 + y__3**2 + y__4**2


# Kinematic Equations

In [35]:
d, a, t, v_i, v_f = Reals('d a t v_i, v_f')
d == v_i * t + (a*t**2)/2
v_f == v_i + a*t

v_f == v_i, + a*t

In [36]:
v_f

v_f

**Problem 1**  
Ima Hurryin is approaching a stoplight moving with a velocity of 30.0 m/s. The light turns yellow, and Ima applies the brakes and skids to a stop. If Ima's acceleration is -8.00 m/s2, then determine the displacement of the car during the skidding process.

In [38]:
d, a, t, v_i, v_f = Reals('d a t v_i, v_f')

equations = [
    d == v_i * t + (a*t**2)/2,
    v_f == v_i + a*t
]
print("Kinematic equations:")
print(equations)

# Given v_i, v_f, and a, find d
problem = [
    v_i == 30,
    v_f == 0,
    a   == -8,
]
print("Problem:")
print(problem)

print("Solution:")
solve(equations + problem)

Kinematic equations:
[d == v_i,*t + (a*t**2)/2, v_f == v_i, + a*t]
Problem:
[v_i, == 30, v_f == 0, a == -8]
Solution:
[a = -8, v_f = 0, v_i, = 30, t = 15/4, d = 225/4]


**Problem 2**  
Ben Rushin is waiting at a stoplight. When it finally turns green, Ben accelerated from rest at a rate of a 6.00 m/s2 for a time of 4.10 seconds. Determine the displacement of Ben's car during this time period.

In [39]:
d, a, t, v_i, v_f = Reals('d a t v_i, v_f')

equations = [
    d == v_i * t + (a*t**2)/2,
    v_f == v_i + a*t
]

# Given v_i, t, and a, find d
problem = [
    v_i == 0,
    t   == 4.10,
    a   == 6,
]

print("Solution:")
solve(equations + problem)

set_option(rational_to_decimal=True)
solve(equations + problem)

Solution:
[a = 6, t = 41/10, v_i, = 0, v_f = 123/5, d = 5043/100]
[a = 6, t = 4.1, v_i, = 0, v_f = 24.6, d = 50.43]


# Bit Tricks

These are some bit-level hacks used for very quick evaluation

**Power to 2**

In [40]:
# x is a power of two if and onnly if
# x != 0 && !(x & (x-1)) is true

x = BitVec('x', 32)
powers = [2**i for i in range(32)]
fast = And(x != 0, x & (x-1) == 0)
slow = Or([x == p for p in powers])

print(fast)
prove(fast == slow)

print("trying to prove bugged version...")
fast = x & (x-1) == 0
prove(fast == slow)

And(x != 0, x & x - 1 == 0)
proved
trying to prove bugged version...
failed to prove


**Oppostie Signs**

In [44]:
# integers x and y have opposite signs iff
# (x ^ y) < 0

x = BitVec('x', 32)
y = BitVec('y', 32)

trick = (x ^ y) < 0

# more straightforward check for opposite signs
opposite = Or(And(x <  0, y >= 0),
              And(x >= 0,  y < 0))
    
prove(trick == opposite)

proved


# Puzzles

**Dog, Cat, and Mouse**  
Consider the following puzzle. Spend exactly 100 dollars and buy exactly 100 animals. Dogs cost 15 dollars, cats cost 1 dollar, and mice cost 25 cents each. You have to buy at least one of each. How many of each should you buy?

In [51]:
dog, cat, mouse = Ints('dog cat mouse')
solve(dog >= 1,
      cat >= 1,
      mouse >= 1,
      dog + cat + mouse == 100,
      1500*dog + 100*cat + 25*mouse == 10000)

[mouse = 56, cat = 41, dog = 3]


**Sudoku**

In [66]:
# 9x9 matrix of integer variables
X = [[Int("x_%s_%s" % (i+1, j+1)) 
      for j in range(9)] for i in range(9)]

# each cell contains a value in [1..9]
cells_c = [And(1 <= X[i][j], X[i][j] <= 9) 
           for i in range(9) for j in range(9)]

# each row contains each digit at most once
rows_c = [Distinct(X[i]) for i in range(9)]

# each column contains each digit at most once
cols_c = [Distinct([X[i][j] for i in range(9)]) 
          for j in range(9)]

# each 3x3 square contains each digit at most once
sq_c   = [Distinct([X[3*i0 + i][3*j0 + j]
                    for i in range (3) for j in range(3)])
          for i0 in range(3) for j0 in range(3)]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, 0 represents empty cell
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

# copy filled in cells into X
instance_c = [If(instance[i][j] == 0,
                 True,
                 X[i][j] == instance[i][j])
              for i in range(9) for j in range(9)]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [[m.evaluate(X[i][j]) for j in range (9)]
         for i in range(9)]
    print_matrix(r)
else:
    print("failed to solve")
    


# Let's remove 9 from the first row and see if there is more than one solution
instance = ((0,0,0,0,0,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

# copy filled in cells into X
instance_c = [If(instance[i][j] == 0,
                 True,
                 X[i][j] == instance[i][j])
              for i in range(9) for j in range(9)]

def n_solutions(n):
    s = Solver()
    s.add(sudoku_c + instance_c)
    i = 0
    while s.check() == sat and i < n:
        m = s.model()
        print_matrix([[m.evaluate(X[i][j]) for j in range (9)]
               for i in range(9)])
        fml = And([X[i][j] == m.evaluate(X[i][j]) for i in range(9) for j in range(9)])
        s.add(Not(fml))
        i += 1
        
print("\nSecond board:")
n_solutions(10)

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]

Second board:
[[6, 1, 7, 8, 2, 4, 9, 3, 5],
 [2, 3, 4, 5, 1, 9, 6, 8, 7],
 [5, 8, 9, 7, 3, 6, 1, 4, 2],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [7, 6, 8, 2, 9, 1, 4, 5, 3],
 [1, 5, 2, 4, 8, 3, 7, 9, 6],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 1, 3, 6, 5, 8, 7, 4],
 [8, 4, 5, 9, 7, 2, 3, 6, 1]]
[[7, 1, 5, 6, 8, 4, 9, 3, 2],
 [2, 3, 4, 5, 1, 9, 8, 6, 7],
 [6, 8, 9, 3, 2, 7, 1, 4, 5],
 [3, 9, 7, 4, 5, 6, 2, 1, 8],
 [4, 6, 8, 2, 9, 1, 7, 5, 3],
 [1, 5, 2, 7, 3, 8, 6, 9, 4],
 [8, 7, 6, 1, 4, 3, 5, 2, 9],
 [9, 2, 3, 8, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]
[[7, 1, 6, 8, 2, 4, 9, 3, 5],
 [2, 3, 4, 5, 1, 9, 6, 8, 7],
 [5, 8, 9, 6, 3, 7, 1, 4, 2],
 [3, 9, 7, 4, 5, 6, 2, 1, 8],
 [4, 6, 8, 2, 9, 1, 7, 5, 3],
 [1, 5, 2, 7, 8, 3, 4, 9,

**Eight Queens**  
The eight queens puzzle is the problem of placing eight chess queens on an 8x8 chessboard so that no two queens attack each other. Thus, a solution requires that no two queens share the same row, column, or diagonal.

In [ ]:
# we know each queen must be in a different row
# so we represent each queen by its column number
Queens = [Int('Q_%i' % (i+1)) for i in range(8)]

# each queen is in a column in range [1..8]
val_c = [And(1 <= Queens[i], Queens[i] <= 8) for i in range(8)]

# at most one queen per column
col_c = [Distinct(Queens)]

# Diagnol constraint
diag_c = [If(i == j,
             True,
             And(Queens[i] - Queens[j] != i - j, Queens[i] - Queens[j] != j - i))
          for i in range(8) for j in range(i)]

solve(val_c + col_c + diag_c)

[Q_3 = 4,
 Q_1 = 5,
 Q_7 = 6,
 Q_8 = 1,
 Q_5 = 3,
 Q_4 = 7,
 Q_2 = 2,
 Q_6 = 8]


# Morloc Type Constraints

Given the Haskell code below with constraints on roll and max, find the constraints on the codomain of rollAdvantage

In [117]:
x, a, b = Ints('x a b')
p, q = Ints('p q')

roll = Function('roll', IntSort(), IntSort())
max = Function('max', IntSort(), IntSort(), IntSort())

roll_c = [x > 0,
          roll(x) > 0,
          roll(x) <= x]

max_c = [Or(And(a > b, max(a, b) == a),
           (And(b >= a, max(a, b) == b)))]

solve(roll_c)
solve(max_c)

s = Solver()
s.add(max_c + roll_c + max_implies_c)
print(s.check())

m = s.model()
print("x                       = ", m.eval(x))
print("roll(20)                = ", m.eval(roll(20)))
print("max(1, 2)               = ", m.eval(max(1, 2)))
print("max(roll(20), roll(20)) = ", m.eval(max(roll(20), roll(20))))

[x = 1, roll = [else -> 1]]
[b = 0, a = 1, max = [else -> 1]]
sat
x                       =  7720
roll(20)                =  1
max(1, 2)               =  1
max(roll(20), roll(20)) =  1


Above is a very badly 